Author(s): Piyush Amitabh

Details: this code converts tiff images to png compressed images, while preserving the folder structure. 
Created: July 15, 2022

License: GNU GPL v3.0

[Oct26, 22] changed to work with image stacks

[Oct27, 22] add metadata reading and writing functionality

[Oct31, 22] by default metadata isn't copies to pngs as imagej can't read it. automated naming scheme based on image axes.

In [1]:
import numpy as np
# from tqdm import tqdm
import xarray as xr
import os
import tifffile as tiff
from skimage import io
import matplotlib.pyplot as plt

In [2]:
import shutil
from datetime import datetime

In [3]:
from PIL import Image
from PIL.PngImagePlugin import PngInfo

# Batchprocess

In [4]:
now = datetime.now()

dt_string = now.strftime("%m%d%Y_%H%M%S")

In [5]:
source_path = input('Enter *Source* directory:\n')
source_path = source_path+'\\' if source_path[-1] != '\\' else source_path #standardize path names
source_dir = source_path.split('\\')[-2]
dest_dir = input('Enter *Destination* directory:\n')
dest_dir = dest_dir+'\\' if dest_dir[-1] != '\\' else dest_dir #standardize path names

default_name = source_dir+'_png_'+dt_string
dest_name = input('Enter Copied folder name (none for default):\n') or default_name
dest_path = dest_dir + dest_name + '\\'
print(f'Images will be compressed and copied to:\n{dest_path}')

Images will be compressed and copied to:
/home/piyush/Downloads/DATA/test/f1B_png_10312022_152259/


In [6]:
#copy all files except the tiff image files
shutil.copytree(source_path,
                dest_path,
                ignore=shutil.ignore_patterns('*.tiff', 'tmp*', '*.tif'))

print(f'Directory structure and all non-image files successfully copied from \nsource:{source_path} to \ndestination:{dest_path}')

Directory structure and all non-image files successfully copied from 
source:/home/piyush/Downloads/DATA/compression test/f1B/ to 
destination:/home/piyush/Downloads/DATA/test/f1B_png_10312022_152259/


In [7]:
def get_tiff_metadata(tif_img_loc):
    with tiff.TiffFile(tif_img_loc) as tif_img:
        for page in tif_img.pages:
            for tag in page.tags:
                tag_name, tag_value = tag.name, tag.value
    # str_axes = tif_img.series[0].axes #get axes order
    return(tag_value) #only return the tag_value as tag_name is just the source of creation (e.g. 'MicroManagerMetadata')

In [8]:
def copy_metadata(tag_value, png_img):
    targetImage = Image.open(png_img)

    metadata = PngInfo()

    for i, j in tag_value.items():
        metadata.add_text(str(i), str(j))

    targetImage.save(png_img, pnginfo=metadata)
    # targetImage = Image.open("\\home\\piyush\\Downloads\\DATA\\test\\"+og_name+"_wm.png")

    # print(targetImage.text)

In [11]:
def single_img_selector(read_image, filepath, c_i, t_i, z_i):
    '''This image selector can select one image from a max. stack of channel, time and z'''
    with tiff.TiffFile(filepath) as multi_tif:
        str_axes = multi_tif.series[0].axes.casefold() #get axes order

    read_image_xarr = xr.DataArray(data=read_image, dims=list(str_axes))

    if ('c' in str_axes) and ('t' in str_axes) and ('z' in str_axes): #stack has c, t, and z
        single_img = read_image_xarr.isel(c=c_i, t=t_i, z=z_i)
    elif 'c' in str_axes: #c and z
        single_img = read_image_xarr.isel(c=c_i, z=z_i)
    elif 'c' in str_axes: #t and z
        single_img = read_image_xarr.isel(t=t_i, z=z_i)
    elif 'c' in str_axes: #c and t
        single_img = read_image_xarr.isel(c=c_i, t=t_i)
    elif 'z' in str_axes: #only z
        single_img = read_image_xarr.isel(z=z_i)
    elif 'c' in str_axes: #only c
        single_img = read_image_xarr.isel(c=c_i)
    elif 't' in str_axes: #only t
        single_img = read_image_xarr.isel(t=t_i)
    else:
        return None

    return single_img

In [32]:
for root, subfolders, filenames in os.walk(source_path[:-1]): #root doesn't have consistent name if source_path has '\\'
    for filename in filenames:
        # print(f'Reading: {filename}')
        
        filepath = root + "\\" + filename
        # print(f'Reading: {filepath}')

        filename_list = filename.split('.')
        og_name = filename_list[0] #first of list=name
        ext = filename_list[-1] #last of list=extension

        if ext=="tif" or ext=="tiff": #only compress tiff files (prevents compression of other filetypes)
            print(f'Reading Image: {filepath}')
            read_image = tiff.imread(filepath)
            write_path = (root+'\\').replace(source_path, dest_path)
            tag_value = get_tiff_metadata(tif_img_loc=filepath) #tag value is a dict containing the tiff tag

            if len(read_image.shape)==2: #single images
                #increase compress level(0-9) to gain few percent better compression but much slower
                io.imsave(fname=write_path+"\\"+og_name+".png", arr=read_image, check_contrast=False, plugin='imageio', compress_level=3)
                print(f'Saved: {og_name}.png')

            elif len(read_image.shape)!=2: #image stacks
                time_max = tag_value.get('FrameIndex') + 1 
                z_max = tag_value.get('SliceIndex') + 1
                channel_max = tag_value.get('ChannelIndex') + 1
                pos_max = tag_value.get('PositionIndex') + 1

                for pos_i in range(pos_max):
                    for time_i in range(time_max):
                        for ch_i in range(channel_max):
                            for z_i in range(z_max):
                                single_image = single_img_selector(read_image, filepath, ch_i, time_i, z_i) #pick single image
                                if len(single_image.shape) != 2: #single image must be 2 dimensional
                                    print('Error: Bad Axes label. Skipping image.')
                                    continue
                                else:
                                    new_name = f'{og_name}_channel{ch_i:03d}_position{pos_i:03d}_time{time_i:03d}_z{z_i:03d}' #for mmanager's format use time{time_i:09d}
                                    io.imsave(fname=write_path+"\\"+new_name+".png", arr=single_image, check_contrast=False, plugin='imageio', compress_level=3)
                                    print(f'Saved: {new_name}.png')
                                    # copy_metadata(tag_value=tag_value, png_img=write_path+"\\"+new_name+".png") #use for copying metadata into the image

Reading Image: /home/piyush/Downloads/DATA/compression test/f1B/BF/BF_MMStack_Pos0.ome.tif
Reading Image: /home/piyush/Downloads/DATA/compression test/f1B/zstack1_1/zstack1_1_MMStack_Pos0.ome.tif
Saved: zstack1_1_MMStack_Pos0_channel000_position000_time000_z000.png
Saved: zstack1_1_MMStack_Pos0_channel000_position000_time000_z001.png
Saved: zstack1_1_MMStack_Pos0_channel000_position000_time000_z002.png
Saved: zstack1_1_MMStack_Pos0_channel000_position000_time000_z003.png
Saved: zstack1_1_MMStack_Pos0_channel000_position000_time000_z004.png
Saved: zstack1_1_MMStack_Pos0_channel000_position000_time000_z005.png
Saved: zstack1_1_MMStack_Pos0_channel000_position000_time000_z006.png
Saved: zstack1_1_MMStack_Pos0_channel000_position000_time000_z007.png
Saved: zstack1_1_MMStack_Pos0_channel000_position000_time000_z008.png
Saved: zstack1_1_MMStack_Pos0_channel000_position000_time000_z009.png
Saved: zstack1_1_MMStack_Pos0_channel000_position000_time000_z010.png
Saved: zstack1_1_MMStack_Pos0_chan